# Computational Theory Assessment

## Problem 1: Binary Words and Operations

In [114]:
import numpy as np

def Parity(x, y, z):
    """
    SHA-1 Parity function: x ^ y ^ z.
    
    Computes the bitwise XOR of three 32-bit words.
    This function is used in SHA-1 and returns 1 for each bit position
    where an odd number of the corresponding bits in x, y, z are 1.
    
    Args:
        x: 32-bit unsigned integer
        y: 32-bit unsigned integer
        z: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of x XOR y XOR z
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    result = x ^ y ^ z

    return np.uint32(result)

In [115]:
def test_parity():
    """Test the Parity function with various inputs."""
    
    # Test 1: Identity - all zeros
    assert Parity(0x00000000, 0x00000000, 0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: Single bit set
    assert Parity(0x00000001, 0x00000000, 0x00000000) == 0x00000001, \
        "Single 1 bit should return 1"
    
    # Test 3 All ones
    assert Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 4: Mixed bits test
    assert Parity(0x00000005, 0x00000003, 0x00000001) == 0x00000007, \
        "Mixed bits: 0101 XOR 0011 XOR 0001 = 0111"
    
    # Test 5: 4-bit test
    assert Parity(0b1010, 0b1100, 0b0011) == 0b0101, \
        "Binary: 1010 XOR 1100 XOR 0011 = 0101"
    print("All Parity tests passed")

# Run the tests
test_parity()

All Parity tests passed


In [116]:
def Ch(x, y, z):
    """
    SHA-1 Ch (Choose) function: (x & y) ^ (~x & z).
    
    The Ch function "chooses" between y and z based on the bits of x:
    - If a bit in x is 1, the corresponding bit from y is chosen
    - If a bit in x is 0, the corresponding bit from z is chosen
    
    Args:
        x: 32-bit unsigned integer (selector)
        y: 32-bit unsigned integer (chosen when x bit is 1)
        z: 32-bit unsigned integer (chosen when x bit is 0)
    
    Returns:
        32-bit unsigned integer result of (x & y) ^ (~x & z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # (x AND y) XOR (NOT x AND z)
    result = (x & y) ^ (~x & z)
    
    return np.uint32(result)

In [117]:
def test_ch():
    """Test the Ch function with various inputs."""
    
    # Test 1: All ones in x (choose all from y)
    assert Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF00) == 0x12345678, \
        "When x is all 1s, result should equal y"
    
    # Test 2: All zeros in x (choose all from z)
    assert Ch(0x00000000, 0x12345678, 0xABCDEF00) == 0xABCDEF00, \
        "When x is all 0s, result should equal z"
    
    # Test 3: Alternating pattern
    assert Ch(0xAAAAAAAA, 0xFFFFFFFF, 0x00000000) == 0xAAAAAAAA, \
        "With alternating x bits, should select alternating bits from y and z"
    
    # Test 4: 4-bit test
    assert Ch(0b1100, 0b1010, 0b0101) == 0b1001, \
        "Binary: Ch(1100, 1010, 0101) should equal 1001"
    
    # Test 5: Actual hash values
    expected5 = np.uint32((0x67452301 & 0xEFCDAB89) ^ (~np.uint32(0x67452301) & 0x98BADCFE))
    assert Ch(0x67452301, 0xEFCDAB89, 0x98BADCFE) == expected5, \
        "Calculation with SHA-1 constants should match direct implementation"
    
    print("All Ch tests passed")

# Run the tests
test_ch()

All Ch tests passed


In [118]:
def Maj(x, y, z):
    """
    SHA-1 Maj (Majority) function: (x & y) ^ (x & z) ^ (y & z).
    
    The Maj function returns the majority vote for each bit position:
    - Returns 1 if at least 2 of the 3 corresponding bits are 1
    - Returns 0 if at least 2 of the 3 corresponding bits are 0
    
    Args:
        x: 32-bit unsigned integer
        y: 32-bit unsigned integer
        z: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of (x & y) ^ (x & z) ^ (y & z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # (x AND y) XOR (x AND z) XOR (y AND z)
    result = (x & y) ^ (x & z) ^ (y & z)
    
    return np.uint32(result)

In [119]:
def test_Maj():
    """Test the Maj function with various inputs."""
    
    # Test 1: All zeros
    assert Maj(0x00000000, 0x00000000, 0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Maj(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Two ones, one zero (majority = 1)
    assert Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) == 0xFFFFFFFF, \
        "Two 1s, one 0 should return 1 for each bit"
    
    # Test 4: Two zeros, one one (majority = 0)
    assert Maj(0x00000000, 0x00000000, 0xFFFFFFFF) == 0x00000000, \
        "Two 0s, one 1 should return 0 for each bit"
    
    # Test 5: 4-bit test with mixed pattern
    assert Maj(0b1100, 0b1010, 0b1001) == 0b1000, \
        "Binary: Maj(1100, 1010, 1001) should equal 1000"

    print("All Maj tests passed")

# Run the tests
test_Maj()

All Maj tests passed


In [120]:
def Sigma0(x):
    """
    SHA-256 Sigma0 (Σ₀²⁵⁶) function: ROTR²(x) ^ ROTR¹³(x) ^ ROTR²²(x)
    
    This function performs three right rotations on a 32-bit word and XORs them.
    
    ROTR^n(x) = circular right rotation of x by n positions
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR²(x) ^ ROTR¹³(x) ^ ROTR²²(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 2, 13, and 22 bits
    rotr2 = np.uint32((x >> 2) | (x << 30))   # ROTR²(x)
    rotr13 = np.uint32((x >> 13) | (x << 19)) # ROTR¹³(x)
    rotr22 = np.uint32((x >> 22) | (x << 10)) # ROTR²²(x)
    
    # XOR all three rotations
    result = rotr2 ^ rotr13 ^ rotr22
    
    return np.uint32(result)

In [121]:
def test_Sigma0():
    """Test the Sigma0 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma0(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma0(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit set
    x = 0x00000001
    rotr2 = np.uint32((x >> 2) | (x << 30))   # 0x40000000
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 0x00020000
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 0x00000400
    expected = rotr2 ^ rotr13 ^ rotr22        # 0x40020400
    assert Sigma0(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"
    
    # Test 4: Alternating bit pattern
    assert Sigma0(0xAAAAAAAA) != 0xAAAAAAAA, \
        "Rotation of alternating pattern should change the pattern"
    
    print("All Sigma0 tests passed")

# Run the tests
test_Sigma0()

All Sigma0 tests passed


In [122]:
def Sigma1(x):
    """
    SHA-256 Sigma1 (Σ₁²⁵⁶) function: ROTR⁶(x) ^ ROTR¹¹(x) ^ ROTR²⁵(x)
    
    This function performs three right rotations on a 32-bit word and XORs them.
    
    ROTR^n(x) = circular right rotation of x by n positions
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR⁶(x) ^ ROTR¹¹(x) ^ ROTR²⁵(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 6, 11, and 25 bits
    rotr6 = np.uint32((x >> 6) | (x << 26))   # ROTR⁶(x)
    rotr11 = np.uint32((x >> 11) | (x << 21)) # ROTR¹¹(x)
    rotr25 = np.uint32((x >> 25) | (x << 7))  # ROTR²⁵(x)
    
    # XOR all three rotations
    result = rotr6 ^ rotr11 ^ rotr25
    
    return np.uint32(result)

In [123]:
def test_Sigma1():
    """Test the Sigma1 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma1(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma1(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit set
    x = 0x00000001
    rotr6 = np.uint32((x >> 6) | (x << 26))   # 0x04000000
    rotr11 = np.uint32((x >> 11) | (x << 21)) # 0x00200000
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 0x00000080
    expected = rotr6 ^ rotr11 ^ rotr25        # 0x04200080
    assert Sigma1(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"
    
    print("All Sigma1 tests passed")

# Run the tests
test_Sigma1()

All Sigma1 tests passed


In [124]:
def sigma0(x):
    """
    SHA-256 sigma0 (σ₀²⁵⁶) function: ROTR⁷(x) ^ ROTR¹⁸(x) ^ SHR³(x)
    
    This function performs two right rotations and one right shift on a 32-bit word and XORs them.
    Used in the message schedule of SHA-256.
    
    ROTR^n(x) = circular right rotation of x by n positions
    SHR^n(x) = right shift of x by n positions (no wrap around)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR⁷(x) ^ ROTR¹⁸(x) ^ SHR³(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25))   # ROTR⁷(x)
    rotr18 = np.uint32((x >> 18) | (x << 14)) # ROTR¹⁸(x)
    
    # Right shift by 3 bits (no wrap around)
    shr3 = np.uint32(x >> 3)                  # SHR³(x)
    
    # XOR all three operations
    result = rotr7 ^ rotr18 ^ shr3
    
    return np.uint32(result)

In [125]:
def test_Sigma0():
    """Test the SHA-256 Sigma0 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma0(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma0(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit test
    x = 0x00000001
    rotr2 = np.uint32((x >> 2) | (x << 30))   # 0x40000000
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 0x00080000
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 0x00000400
    expected = rotr2 ^ rotr13 ^ rotr22        # 0x40080400
    assert Sigma0(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"
    
    print("All Sigma0 tests passed")

# Run the tests
test_Sigma0()

All Sigma0 tests passed


In [126]:
def sigma1(x):
    """
    SHA-256 sigma1 (σ₁²⁵⁶) function: ROTR¹⁷(x) ^ ROTR¹⁹(x) ^ SHR¹⁰(x)
    
    This function performs two right rotations and one right shift on a 32-bit word and XORs them.
    Used in the message schedule of SHA-256 (extending the message).
    
    ROTR^n(x) = circular right rotation of x by n positions
    SHR^n(x) = right shift of x by n positions (no wrap around, fills with zeros)
    
    Reference: FIPS 180-4, Section 4.1.2 (SHA-256 Functions)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR¹⁷(x) ^ ROTR¹⁹(x) ^ SHR¹⁰(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 17 and 19 bits
    rotr17 = np.uint32((x >> 17) | (x << 15))  # ROTR¹⁷(x)
    rotr19 = np.uint32((x >> 19) | (x << 13))  # ROTR¹⁹(x)
    
    # Right shift by 10 bits (no wrap around)
    shr10 = np.uint32(x >> 10)                 # SHR¹⁰(x)
    
    # XOR all three operations
    result = rotr17 ^ rotr19 ^ shr10
    
    return np.uint32(result)

In [ ]:
def test_Sigma1():
    """Test the SHA-256 Sigma1 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma1(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma1(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit set
    x = 0x00000001
    rotr6 = np.uint32((x >> 6) | (x << 26))   # 0x04000000
    rotr11 = np.uint32((x >> 11) | (x << 21)) # 0x00200000
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 0x00000080
    expected = rotr6 ^ rotr11 ^ rotr25        # 0x04200080
    assert Sigma1(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"

    print("All Sigma1 tests passed")

# Run the tests
test_Sigma1()

✓ All Sigma1 tests passed


## Problem 2: Fractional Parts of Cube Roots


## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords

# End